In [1]:
import requests
import json
import pickle
import time

In [2]:
#API base url
baseurl="https://api.pushshift.io/reddit/search/submission?size=100&subreddit="
#topics to scrape for
subreddit=["atheism","christianity"]
#mininimum word count per post
word_count_threshold=20
#minimum posts to scrape for
min_posts_required=5000

In [3]:
binary_scrapes=[]

In [4]:
def check(js,current_count,continue_flag,b_url,ss):
    """
    A function which checks how many valid posts are there in the scraped JSON,
    and finds the earliest date to pass to the next scrape.
    """
    valid_posts_retrieved=current_count
    data=js['data']
    
    for l in range(len(js['data'])):
        try:
            selftext=data[l]['selftext']
            len_selftext=len(selftext.split())

            if len_selftext >= word_count_threshold:
                valid_posts_retrieved+=1
                ss.append(selftext)
        except:
            continue
    
    if valid_posts_retrieved>min_posts_required:
        continue_flag=False
    else:
        continue_flag=True
    
    last_epoch=data[-1]['created_utc']
    url=f'{b_url}{last_epoch}'      
    
    print(f"Posts obtained: {valid_posts_retrieved}")
    
    return valid_posts_retrieved, continue_flag, url, saved_scrapes

In [5]:
#iterate through each subreddit topic and save to pickle file
for sub in subreddit:
    b_url=baseurl+sub+"&before="
    url=b_url
    continue_flag=True
    current_count=0
    saved_scrapes=[]
    
    print("----------------")
    print(f"Running subreddit: {sub}")
    
    while continue_flag:
        resp=requests.get(url)
#         print(f"Fetching url: {url}")  #debug purpose only
        js=json.loads(resp.text)
        
        current_count, continue_flag , url, saved_scrapes= check(js,current_count,continue_flag,b_url,saved_scrapes)
        time.sleep(1) #adds a delay to prevent server from DDOS-ed
    
    scrape_path=f"../data/scrapes_{sub}.pickle"
    with open(scrape_path,'wb') as handle:
        pickle.dump(saved_scrapes,handle)
    print(f"Dump file scrapes_{sub}.pickle saved")

----------------
Running subreddit: atheism
Posts obtained: 22
Posts obtained: 74
Posts obtained: 126
Posts obtained: 178
Posts obtained: 222
Posts obtained: 261
Posts obtained: 312
Posts obtained: 359
Posts obtained: 396
Posts obtained: 444
Posts obtained: 500
Posts obtained: 557
Posts obtained: 599
Posts obtained: 641
Posts obtained: 683
Posts obtained: 725
Posts obtained: 775
Posts obtained: 827
Posts obtained: 878
Posts obtained: 924
Posts obtained: 971
Posts obtained: 1016
Posts obtained: 1066
Posts obtained: 1110
Posts obtained: 1160
Posts obtained: 1210
Posts obtained: 1256
Posts obtained: 1301
Posts obtained: 1353
Posts obtained: 1396
Posts obtained: 1435
Posts obtained: 1484
Posts obtained: 1530
Posts obtained: 1577
Posts obtained: 1625
Posts obtained: 1668
Posts obtained: 1715
Posts obtained: 1761
Posts obtained: 1804
Posts obtained: 1850
Posts obtained: 1898
Posts obtained: 1945
Posts obtained: 1985
Posts obtained: 2029
Posts obtained: 2073
Posts obtained: 2115
Posts obtaine